In [1]:
import pymongo
import pandas as pd
from flask import Flask, request, jsonify
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from bson import ObjectId

app = Flask(__name__)

# Connect to MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['LMS']
collection = db['books']

# Fetch data
books = list(collection.find())
df = pd.DataFrame(books)

# Preprocess data
df['features'] = df['desc']   # Use 'desc' for keyword features
vectorizer = TfidfVectorizer()
feature_matrix = vectorizer.fit_transform(df['features'])
cosine_sim = cosine_similarity(feature_matrix, feature_matrix)


# Function to convert ObjectId to string
def convert_objectid_to_str(data):
    if isinstance(data, ObjectId):
        return str(data)
    return data

# Improved recommendation function with self-match included
def get_recommendations(keyword, number, cosine_sim=cosine_sim):
    # Find exact match indices for the keyword in 'desc'
    matching_indices = df[df['desc'].str.contains(keyword, case=False, na=False)].index

    if not matching_indices.empty:
        # If exact match found, proceed with similarity-based recommendations
        idx = matching_indices[0]  # Using the first matching index
        sim_scores = list(enumerate(cosine_sim[idx]))
    else:
        # If no exact match, calculate similarity between keyword and entire 'desc' column
        keyword_vector = vectorizer.transform([keyword])
        sim_scores = list(enumerate(cosine_similarity(keyword_vector, feature_matrix)[0]))

    # Sort and get top recommendations, including self-match
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[:number]  # Get top 10 recommendations including the closest match

    book_indices = [i[0] for i in sim_scores]
    recommendations = df.iloc[book_indices].to_dict(orient='records')

    # Convert ObjectId fields to string
    for rec in recommendations:
        for key in rec.keys():
            rec[key] = convert_objectid_to_str(rec[key])

    return recommendations

@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.get_json()
    keyword = data.get('keyword', '')
    number = data.get('number', '')
    recommendations = get_recommendations(keyword,number)
    return jsonify(recommendations)  # Return full book info in JSON format

if __name__ == '__main__':
    app.run(port=5000)


 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [11/Jul/2025 13:27:44] "POST /recommend HTTP/1.1" 200 -
127.0.0.1 - - [11/Jul/2025 13:29:12] "POST /recommend HTTP/1.1" 200 -
